In [8]:
import os
import zipfile
import requests

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models

from io import BytesIO

In [2]:

# Downloading and extracting the dataset (Cats vs Dogs)
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-E5C8-4431-9AD4-3DF3FF04978E/kagglecatsanddogs_3367a.zip
!unzip -q kagglecatsanddogs_3367a.zip
!ls PetImages


'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.
'ls' is not recognized as an internal or external command,
operable program or batch file.


In [3]:


# Downloading the dataset with the new URL
url = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip'
response = requests.get(url)

# Check if the response is valid and content is a zip file
if response.status_code == 200 and 'zip' in response.headers.get('Content-Type', ''):
    zip_file = zipfile.ZipFile(BytesIO(response.content))
    # Extracting the dataset
    zip_file.extractall('.')
    # Listing the contents of the PetImages directory
    print(os.listdir('PetImages'))
else:
    print("Failed to download a valid zip file. Please check the URL or your internet connection.")



Failed to download a valid zip file. Please check the URL or your internet connection.


In [4]:

# Data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Loading dataset
train_data = torchvision.datasets.ImageFolder(root='PetImages', transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)


In [9]:
# Loading a pretrained model (ResNet18) and modifying it for binary classification
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # Modifying for 2 classes: Cats and Dogs

# Checking if GPU is available and moving the model to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)


NameError: name 'ResNet18_Weights' is not defined

In [6]:

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [7]:

# Training the model
for epoch in range(2):  # Looping over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')


C:\Users\francji1\OneDrive\gpu_test\venv\Lib\site-packages\PIL\TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


[1,   100] loss: 0.170
[1,   200] loss: 0.074
[1,   300] loss: 0.055
[1,   400] loss: 0.058
[1,   500] loss: 0.064
[1,   600] loss: 0.052
[1,   700] loss: 0.045
[2,   100] loss: 0.032
[2,   200] loss: 0.024
[2,   300] loss: 0.026
[2,   400] loss: 0.032
[2,   500] loss: 0.018
[2,   600] loss: 0.027
[2,   700] loss: 0.023
Finished Training
